In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
df_train = pd.read_csv('AWM_train.csv')
df_valid=pd.read_csv('AWM_dev.csv')
df_test=pd.read_csv('AWM_test_without_labels.csv')
df_train


,Text,Class
0,നവ്യയുടെ കയ്യിന്ന് കിട്ടിയതാണല്ലോ. ഇവൾക്ക് അതൊ...,Abusive
1,"""ഇവരുടെ പ്രശ്നം അസൂയ ആണ്, സുന്ദരികൾ ആയ നടിമാരോ...",Abusive
2,ചുമ്മാതല്ല ഇവളെ പണ്ട് ലവർ അലക്കി വിട്ടത്...വായ...,Abusive
3,"""ഒരു സിനിമയിൽ ജഗതി മദാമ്മയായിട്ട് വരുന്നുണ്ടല്...",Non-Abusive
4,ഈ വർഷത്തെ ബൂലോക തോൽവി പരാജയം അതിനുള്ള ഓസ്‌ക്കാ...,Abusive
...,...,...
2928,അടിപൊളി അടിപൊളി എല്ലാർക്കും ഹാപ്പി ആയല്ലോ ഇനി ...,Non-Abusive
2929,എനിക്ക് ഫാമിലിക്കും ഭയങ്കര ഇഷ്ടം ആണ് മോളെ എന്ന...,Non-Abusive
2930,നന്ദികേട് പറയല്ലേ മോളെ നിനക്കുവേണ്ടി റോബിൻ സംസ...,Non-Abusive
2931,"""നല്ല കൂളായിട്ട് തേച്ചു... പാവം ബ്ലസിളി, റോബിൻ""",Abusive


In [4]:
df_valid

,Text,Class
0,ഞാനും എന്റെ ഫാമിലി മോളെ വ്യക്തിത്വവും ഗെയിം കണ...,Non-Abusive
1,സാരമില്ല മോളെ നീ ആരുടെ മുന്നിലും കരയരുത് അത് ...,Non-Abusive
2,ഡോക്ടർക്ക് ഇങ്ങനെ തന്നെ വേണം. ഈ വിവരമില്ലാത്ത...,Abusive
3,"""ട്രോളന്മാർക്കും, യൂട്യൂബ് ചാനൽസ് നും ചാകര ആയി .""",Non-Abusive
4,ഇപ്പോ പറഞ്ഞ നോ നേര്തെ പറഞ്ഞഗിൽ ഇത്തരം മോശം കമന...,Non-Abusive
...,...,...
624,ഒന്ന് ഇറങ്ങി പോടീ അറുവാണി.... അവളെഡ് അമ്മേടെ....,Abusive
625,50 ലക്ഷം കയ്യിൽ വെക്കാൻ ഒരു യോഗ്യതയും ഇല്ല,Non-Abusive
626,സ്വന്തമായി നിലപാട് ഇല്ലേ... എല്ലാം സൂരജ് പറഞ്ഞ...,Non-Abusive
627,പറഞ്ഞ സ്വതിയ്ക്ക് ഇനിക്ക് പൈസ അയച്ച് തരണം,Non-Abusive


In [5]:
df_test

,id,Text
0,1,സൂരജ് നിന്റെ ആര് ആണ് ആള് ഇറക്കി പേടിപ്പിക്കുക ആണോ
1,2,എത്ര അലക്കി വെളുപ്പിച്ചാലും നിനക്കു അർഹത ഇല്ലാ...
2,3,50 ലക്ഷം കയ്യിൽ വയ്ക്കാൻ ഒരിക്കലും യോഗ്യത ഇല്ല...
3,4,"""ബിഗ് ബോസ്സിൽ നിങ്ങളുടെ അഭിനയം എന്തായിരുന്നു,മ..."
4,5,അത് അങ്ങനെയാ നമ്മുടെ ഉള്ളിൽ നന്മ ഉണ്ടെങ്കിൽ പട...
...,...,...
624,625,ഞാൻ ചെയ്ത വോട്ട് പാഴായില്ല. വളരെ സന്തോഷം
625,626,സ്വന്തം കഴിവ് കൊണ്ടല്ല ജയിച്ചത് എന്നറിയുന്ന ദി...
626,627,റോബിൻ കഴിഞ്ഞാൽ ദിൽഷ തന്നെയാണ് ഈ ട്രോഫിക് അ...
627,628,റോബിനെ കെട്ടണം. ഇല്ലെങ്കിൽ വോട്ടിട്ട അന്തം ഫാൻ...


In [6]:
def preprocess_text(text, lang='ml'):
    normalizer = IndicNormalizerFactory().get_normalizer(lang)
    text = normalizer.normalize(text)
    tokens = list(indic_tokenize.trivial_tokenize(text, lang))
    return ' '.join(tokens).lower()

df_train['Text'] = df_train['Text'].apply(lambda x: preprocess_text(x, lang='ml'))
df_valid['Text'] = df_valid['Text'].apply(lambda x: preprocess_text(x, lang='ml'))
df_test['Text'] = df_valid['Text'].apply(lambda x: preprocess_text(x, lang='ml'))


In [7]:
X_train=df_train['Text']
X_valid=df_valid['Text']
X_test=df_test['Text']
Y_train=df_train['Class']
Y_valid=df_valid['Class']



In [8]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_valid_vec = vectorizer.transform(X_valid)
X_test_vec=vectorizer.transform(X_test)

In [9]:
model=SVC(probability=True,C=10,kernel='rbf')
model.fit(X_train_vec, Y_train)

SVC(C=10, probability=True)

In [10]:
Y_valid_predicted=model.predict(X_valid_vec)


In [11]:
print("Classification Report:\n", classification_report(Y_valid, Y_valid_predicted))
print("Confusion Matrix:\n", confusion_matrix(Y_valid, Y_valid_predicted))
precision = precision_score(Y_valid, Y_valid_predicted, average='weighted')
recall = recall_score(Y_valid, Y_valid_predicted, average='weighted')
f1 = f1_score(Y_valid, Y_valid_predicted, average='weighted')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Classification Report:
               precision    recall  f1-score   support

     Abusive       0.63      0.64      0.64       303
 Non-Abusive       0.66      0.65      0.66       326

    accuracy                           0.65       629
   macro avg       0.65      0.65      0.65       629
weighted avg       0.65      0.65      0.65       629

Confusion Matrix:
 [[195 108]
 [113 213]]
Precision: 0.65
Recall: 0.65
F1-Score: 0.65


In [12]:
Y_predicted=model.predict(X_test_vec)

In [13]:
Y_predicted

array(['Non-Abusive', 'Abusive', 'Abusive', 'Abusive', 'Non-Abusive',
       'Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive', 'Non-Abusive',
       'Abusive', 'Non-Abusive', 'Abusive', 'Non-Abusive', 'Abusive',
       'Non-Abusive', 'Non-Abusive', 'Non-Abusive', 'Non-Abusive',
       'Abusive', 'Non-Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Non-Abusive', 'Abusive', 'Abusive', 'Non-Abusive',
       'Non-Abusive', 'Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive', 'Abusive',
       'Abusive', 'Non-Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive',
       'Non-Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive', 'Abusive',
       'Abusive', 'Abusive', 'Non-Abusive', 'Non-Abusive', 'Non-Abusive',
       'Abusive', 'Non-Abusive', 'Abusive', 'Non-Abusive', 'Abusive',
       'Non-Abusive', 'Non-

In [14]:
import joblib


joblib.dump(model, 'malayalam_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

print("Model saved successfully as 'malayalam_model.pkl'")
print("Vectorizer saved successfully as 'tfidf_vectorizer.pkl'")


Model saved successfully as 'malayalam_model.pkl'
Vectorizer saved successfully as 'tfidf_vectorizer.pkl'


In [17]:
predicted=pd.read_csv('Syndicate_IIITK_malayalam.csv')
predicted

,id,Class
0,1,Abusive
1,2,Abusive
2,3,Abusive
3,4,Abusive
4,5,Non-Abusive
...,...,...
624,625,Non-Abusive
625,626,Non-Abusive
626,627,Non-Abusive
627,628,Abusive
